In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip -q install langchain huggingface_hub openai==0.27.2 google-search-results tiktoken cohere google-cloud-aiplatform

In [3]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [4]:
import os
from getpass import getpass

OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = os.environ['AZURE_API_VERSION']
OPENAI_API_BASE = os.environ['AZURE_OPENAI_BASE']
OPENAI_API_KEY = os.environ['AZURE_OPENAI_KEY']
DEPLOYMENT_NAME = os.environ['DEPLOYMENT_NAME']

os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
# os.environ["COHERE_API_KEY"] = getpass(prompt = "Enter you Cohere API KEY")
os.environ["COHERE_API_KEY"] = os.environ["COHERE_KEY"]
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.environ["HUGGINGFACE_TOKEN"]
os.environ["GOOGLE_API_KEY"] = os.environ["PALM_API_KEY"]

In [5]:
import openai

openai.api_type = "azure"
openai.api_key = os.environ["AZURE_OPENAI_KEY"]
openai.api_base = os.environ["AZURE_OPENAI_BASE"]
openai.api_version = "2023-05-15"

In [6]:
pip show langchain

Name: langchainNote: you may need to restart the kernel to use updated packages.

Version: 0.0.309
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\Users\Harshu\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: aiohttp, anyio, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [7]:
overall_temperature = 0.1

In [8]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.environ["HUGGINGFACE_TOKEN"]

## **Setting up LLMs**

In [9]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [10]:
flan_20B = HuggingFaceHub(
    repo_id = "google/flan-ul2",
    model_kwargs = {
        "temperature" : overall_temperature,
        "max_new_tokens" : 200
    }
)

c:\Users\Harshu\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Harshu\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [11]:
flan_t5xxl = HuggingFaceHub(repo_id="google/flan-t5-xxl", 
                         model_kwargs={"temperature":overall_temperature, 
                                       "max_new_tokens":200}
                         ) 

In [12]:

gpt_j6B = HuggingFaceHub(repo_id="EleutherAI/gpt-j-6B", 
                         model_kwargs={"temperature":overall_temperature, 
                                       "max_new_tokens":100}
                         )

## **Setting up Google Palm model**

In [13]:
from langchain.llms import GooglePalm

In [14]:
palm = GooglePalm()

## **Setting up Azure OpenAI model**

In [15]:
os.environ['OPENAI_API_TYPE'] = OPENAI_API_TYPE
os.environ['OPENAI_API_VERSION'] = OPENAI_API_VERSION
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['OPENAI_API_BASE'] = OPENAI_API_BASE

In [16]:
from langchain.llms import AzureOpenAI, OpenAI, OpenAIChat
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI

In [17]:
azure_openai = llm = AzureOpenAI(
    deployment_name="Developer-35-turbo-instruct",
    model_name="gpt-35-turbo-instruct",
)

chatGPT_turbo = OpenAIChat(
             engine="Developer-35-turbo",
             model_name="gpt-35-turbo",
             temperature=overall_temperature, 
             max_tokens = 256,
             )

c:\Users\Harshu\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\llms\openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


## **Setting up Cohere models**

In [18]:
from langchain.llms import Cohere

In [29]:
cohere_command_x1 = Cohere(
    cohere_api_key=os.environ["COHERE_API_KEY"],
    model = "command-xlarge",
    temperature = overall_temperature,
    max_tokens = 500
)

In [30]:
cohere_command_x1_nightly = Cohere(
    cohere_api_key=os.environ["COHERE_API_KEY"],
    model = "command-xlarge-nightly",
    temperature = overall_temperature,
    max_tokens = 500
)

## **Setup a comparison lab**

In [21]:
from langchain.model_laboratory import ModelLaboratory

In [22]:
from langchain.prompts import PromptTemplate

In [23]:
template = """ Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template = template, input_variables=["question"])

In [32]:
lab = ModelLaboratory.from_llms([
    azure_openai, 
    chatGPT_turbo,
    palm,
    # gpt_j6B, 
    flan_20B,
    flan_t5xxl, 
    cohere_command_x1, 
    cohere_command_x1_nightly
], prompt = prompt)

In [25]:
lab.compare("What is the opposite of up?")

Input:
What is the opposite of up?

AzureOpenAI
Params: {'deployment_name': 'Developer-35-turbo-instruct', 'model_name': 'gpt-35-turbo-instruct', 'temperature': 0.7, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}
 The opposite of up is down. However, this is not the only opposite. We can also say the opposite of up is down, or the opposite of up is horizontal. This is because up and down are directions, while horizontal is a position. Another possible opposite is top, as in the top of a mountain, compared to the bottom or base of the mountain. Similarly, we could say the opposite of up is low, as in a low point compared to a high point. Ultimately, the opposite of up can have multiple interpretations and is dependent on the context in which it is used.

OpenAIChat
Params: {'model_name': 'gpt-35-turbo', 'engine': 'Developer-35-turbo', 'temperature': 0.1, 'max_tokens': 256}
The opposite of up is down.

Goog

In [26]:
lab.compare("Answer the following question by reasoning step by step. The cafeteria had 23 apples. \
If they used 20 for lunch, and bought 6 more, how many apple do they have?")

Input:
Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?

AzureOpenAI
Params: {'deployment_name': 'Developer-35-turbo-instruct', 'model_name': 'gpt-35-turbo-instruct', 'temperature': 0.7, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}
 

Step 1: The cafeteria had 23 apples.
This means that initially, the cafeteria had 23 apples.

Step 2: If they used 20 for lunch,
This means that out of the 23 apples, 20 were used for lunch.

Step 3: and bought 6 more,
This means that in addition to the 3 apples left, the cafeteria bought 6 more apples.

Step 4: how many apples do they have?
To find the total number of apples the cafeteria has now, we need to add the 3 apples left with the 6 apples bought, which gives us a total of 9 apples.

Step 5: Therefore, the cafeteria now has 9 apples. 
We can conclude tha

In [33]:
lab.compare("How to make pizza?")

Input:
How to make pizza?

AzureOpenAI
Params: {'deployment_name': 'Developer-35-turbo-instruct', 'model_name': 'gpt-35-turbo-instruct', 'temperature': 0.7, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}
 First, gather all the necessary ingredients: pizza dough, tomato sauce, cheese, toppings of your choice (such as pepperoni, vegetables, or pineapple), and any desired herbs or seasonings.

Step 1: Preheat your oven to 450 degrees Fahrenheit (232 degrees Celsius).

Step 2: Roll out your pizza dough on a floured surface to the desired thickness. You can also use a pre-made pizza crust if you prefer.

Step 3: Spread a thin layer of tomato sauce onto the dough, leaving about half an inch around the edge.

Step 4: Sprinkle your desired amount of cheese on top of the sauce. You can use any kind of cheese, such as mozzarella, cheddar, or a blend of cheeses.

Step 5: Add your desired toppings on top of the chees